In [1]:
# imports
from astropy.table import QTable
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const
import os
import math
import shutil
import pdb
import subprocess
import re
%matplotlib inline

In [100]:
n = 100

# b.in file constants
name_b = 'b'
modules_b = 'atmesc'

dMass = -2
dObliquity = 23.5
dRadGyra = 0.5

dEcc = 0.0
dSemi = np.linspace(0.01,1,n)

bHaltEnvelopeGone = 1
dEnvelopeMass = -1
dAtmXAbsEffH = np.linspace(0.01,0.5,n)

saOutputOrder_b = 'Time -Mass -EnvelopeMass'

# star.in file constants
name_star = 'star'
modules_star = 'stellar'
dMass_star = 1.0
#dRadGyra_star = 0.5
#dRotPeriod_star = -24


saOutputOrder_star = 'Time'

#vpl.in file constants
sys_name = 'sys'
iVerbose = 5
bOverwrite = 1

saBodyFiles = 'star.in b.in'

sUnitMass = 'solar'
sUnitLength = 'aU'
sUnitTime = 'YEARS'
sUnitAngle = 'd'
sUnitTemp = 'K'

bDoLog = 1
iDigits = 6
dMinValue = 1e-10

bDoForward = 1
bVarDt = 1
dEta = 0.01
dStopTime = 10e9
dOutputTime = 1e9



In [95]:
# now to write some files
# this is for varying semi and efficiency

runs_dir = '/home/dm1681/Research/Research/Merging_Perts/new_atm/'
n = 0
for semi in dSemi:
    for xabs in dAtmXAbsEffH:
        name_idx = '%05i'%n

        if os.path.isdir(runs_dir+name_idx) == True: #sim folders exist
            shutil.rmtree(runs_dir) #removes previous folders; THIS LINE DELETES THE ENTIRE RUNS DIRECTORY
            os.makedirs(runs_dir+name_idx)
        else: #sims folder does not exist
            os.makedirs(runs_dir+name_idx)
            
        # b.in strings
        mass_str = str(dMass)
        obl_str = str(dObliquity)
        radgyr_str = str(dRadGyra)
        ecc_str = str(dEcc)
        semi_str = str(semi)
        haltenv_str = str(bHaltEnvelopeGone)
        envmass_str = str(dEnvelopeMass)
        xabs_str = str(xabs)
        
        # star.in strings
        mass_star_str = str(dMass_star)
        radgyra_star_str = str(dRadGyra_star)
        rotper_star_str = str(dRotPeriod_star)
        
        # vpl.in strings
        iVerbose_str = str(iVerbose)
        bOverwrite_str = str(bOverwrite)
        bDoLog_str = str(bDoLog)
        iDigits_str = str(iDigits)
        dMinValue_str = str(dMinValue)
        bDoForward_str = str(bDoForward)
        bVarDt_str = str(bVarDt)
        dEta_str = str(dEta)
        dStopTime_str = str(dStopTime)
        dOutputTime_str = str(dOutputTime)
        
        # write b.in
        b = open(runs_dir+name_idx+'/b.in','w')
        b_content = ('sName\t\t'+ name_b + 
                     '\nsaModules\t'+modules_b+
                     '\n\ndMass\t\t'+mass_str+
                     '\ndObliquity\t'+obl_str+
                     '\ndRadGyra\t'+radgyr_str+
                     '\n\ndEcc\t\t'+ecc_str+
                     '\ndSemi\t\t'+semi_str+
                     '\n\nbHaltEnvelopeGone\t'+haltenv_str+
                     '\ndEnvelopeMass\t\t'+envmass_str+
                     '\ndAtmXAbsEffH\t'+xabs_str+
                     '\n\nsaOutputOrder\t'+saOutputOrder_b+'\n')
        b.write(b_content)
        b.close()
        
        # write star.in
        star = open(runs_dir+name_idx+'/star.in','w')
        star_content = ('sName\t\t'+name_star+
                        '\nsaModules\t'+modules_star+
                        '\n\ndMass\t\t'+mass_star_str+
                        #'\ndRotPeriod\t'+rotper_star_str+
                        #'\ndRadGyra\t'+radgyra_star_str+
                        '\n\nsaOutputOrder\t'+saOutputOrder_star+'\n')
        star.write(star_content)
        star.close()
        
        vpl = open(runs_dir+name_idx+'/vpl.in','w')
        vpl_content = ('sSystemName\t'+sys_name+
                       '\niVerbose\t'+iVerbose_str+
                       '\nbOverwrite\t'+bOverwrite_str+
                       '\n\nsaBodyFiles\t'+saBodyFiles+
                       '\n\nsUnitMass\t'+sUnitMass+
                       '\nsUnitLength\t'+sUnitLength+
                       '\nsUnitTime\t'+sUnitTime+
                       '\nsUnitAngle\t'+sUnitAngle+
                       '\nsUnitTemp\t'+sUnitTemp+
                       '\n\nbDoLog\t\t'+bDoLog_str+
                       '\niDigits\t\t'+iDigits_str+
                       '\ndMinValue\t'+dMinValue_str+
                       '\n\nbDoForward\t'+bDoForward_str+
                       '\nbVarDt\t\t'+bVarDt_str+
                       '\ndEta\t\t'+dEta_str+
                       '\ndStopTime\t'+dStopTime_str+
                       '\ndOutputTime\t'+ dOutputTime_str+'\n')
        vpl.write(vpl_content)
        vpl.close()
        
        n+=1
        
        
    

In [102]:
# now we want to see when they lose their envelope as a function of semi
# for a specified efficieny = 0.15

runs_dir = '/home/dm1681/Research/Research/Merging_Perts/var_semi_const_eff/'
t = 10000
dAtmXAbsEffH = 0.15
dSemi = np.linspace(0.01,1,t)
n = 0
for semi in dSemi:
    name_idx = '%05i'%n

    if os.path.isdir(runs_dir+name_idx) == True: #sim folders exist
        shutil.rmtree(runs_dir) #removes previous folders; THIS LINE DELETES THE ENTIRE RUNS DIRECTORY
        os.makedirs(runs_dir+name_idx)
    else: #sims folder does not exist
        os.makedirs(runs_dir+name_idx)

    # b.in strings
    mass_str = str(dMass)
    obl_str = str(dObliquity)
    radgyr_str = str(dRadGyra)
    ecc_str = str(dEcc)
    semi_str = str(semi)
    haltenv_str = str(bHaltEnvelopeGone)
    envmass_str = str(dEnvelopeMass)
    xabs_str = str(xabs)

    # star.in strings
    mass_star_str = str(dMass_star)
    radgyra_star_str = str(dRadGyra_star)
    rotper_star_str = str(dRotPeriod_star)

    # vpl.in strings
    iVerbose_str = str(iVerbose)
    bOverwrite_str = str(bOverwrite)
    bDoLog_str = str(bDoLog)
    iDigits_str = str(iDigits)
    dMinValue_str = str(dMinValue)
    bDoForward_str = str(bDoForward)
    bVarDt_str = str(bVarDt)
    dEta_str = str(dEta)
    dStopTime_str = str(dStopTime)
    dOutputTime_str = str(dOutputTime)

    # write b.in
    b = open(runs_dir+name_idx+'/b.in','w')
    b_content = ('sName\t\t'+ name_b + 
                 '\nsaModules\t'+modules_b+
                 '\n\ndMass\t\t'+mass_str+
                 '\ndObliquity\t'+obl_str+
                 '\ndRadGyra\t'+radgyr_str+
                 '\n\ndEcc\t\t'+ecc_str+
                 '\ndSemi\t\t'+semi_str+
                 '\n\nbHaltEnvelopeGone\t'+haltenv_str+
                 '\ndEnvelopeMass\t\t'+envmass_str+
                 '\ndAtmXAbsEffH\t'+xabs_str+
                 '\n\nsaOutputOrder\t'+saOutputOrder_b+'\n')
    b.write(b_content)
    b.close()

    # write star.in
    star = open(runs_dir+name_idx+'/star.in','w')
    star_content = ('sName\t\t'+name_star+
                    '\nsaModules\t'+modules_star+
                    '\n\ndMass\t\t'+mass_star_str+
                    #'\ndRotPeriod\t'+rotper_star_str+
                    #'\ndRadGyra\t'+radgyra_star_str+
                    '\n\nsaOutputOrder\t'+saOutputOrder_star+'\n')
    star.write(star_content)
    star.close()

    vpl = open(runs_dir+name_idx+'/vpl.in','w')
    vpl_content = ('sSystemName\t'+sys_name+
                   '\niVerbose\t'+iVerbose_str+
                   '\nbOverwrite\t'+bOverwrite_str+
                   '\n\nsaBodyFiles\t'+saBodyFiles+
                   '\n\nsUnitMass\t'+sUnitMass+
                   '\nsUnitLength\t'+sUnitLength+
                   '\nsUnitTime\t'+sUnitTime+
                   '\nsUnitAngle\t'+sUnitAngle+
                   '\nsUnitTemp\t'+sUnitTemp+
                   '\n\nbDoLog\t\t'+bDoLog_str+
                   '\niDigits\t\t'+iDigits_str+
                   '\ndMinValue\t'+dMinValue_str+
                   '\n\nbDoForward\t'+bDoForward_str+
                   '\nbVarDt\t\t'+bVarDt_str+
                   '\ndEta\t\t'+dEta_str+
                   '\ndStopTime\t'+dStopTime_str+
                   '\ndOutputTime\t'+ dOutputTime_str+'\n')
    vpl.write(vpl_content)
    vpl.close()

    n+=1
        

In [ ]:
# what if we wanted to validate the findings of Lehmer?
# we want to show that there is a transition between rockey and gaseous at 1.7 earthRads within 100Myrs
# we want to vary 3 parameters: Radius, atm mass ratio, efficiency.
# for 10000 runs, thats 21 values
# for 1,000,000 runs, thats 100 values

n = 100

# b.in file constants
name_b = 'b'
modules_b = 'atmesc'

dRadius = np.linspace(0.5,4.0,n)
dMass = -2 # to be determined
dObliquity = 23.5
dRadGyra = 0.5

dEcc = 0.0
dSemi = 0.1

bHaltEnvelopeGone = 1
#dEnvelopeMass = -1
dXFrac = np.linspace(0.01,0.5,n)
dAtmXAbsEffH = np.linspace(0.01,0.5,n)

saOutputOrder_b = 'Time -Mass -EnvelopeMass'

# star.in file constants
name_star = 'star'
modules_star = 'stellar'
dMass_star = 1.0
#dRadGyra_star = 0.5
#dRotPeriod_star = -24


saOutputOrder_star = 'Time'

#vpl.in file constants
sys_name = 'sys'
iVerbose = 5
bOverwrite = 1

saBodyFiles = 'star.in b.in'

sUnitMass = 'solar'
sUnitLength = 'aU'
sUnitTime = 'YEARS'
sUnitAngle = 'd'
sUnitTemp = 'K'

bDoLog = 1
iDigits = 6
dMinValue = 1e-10

bDoForward = 1
bVarDt = 1
dEta = 0.01
dStopTime = 100e6
dOutputTime = 10e6


In [ ]:
    
    name_idx = '%07i'%n

    if os.path.isdir(runs_dir+name_idx) == True: #sim folders exist
        shutil.rmtree(runs_dir) #removes previous folders; THIS LINE DELETES THE ENTIRE RUNS DIRECTORY
        os.makedirs(runs_dir+name_idx)
    else: #sims folder does not exist
        os.makedirs(runs_dir+name_idx)

    # b.in strings
    mass_str = str(dMass)
    obl_str = str(dObliquity)
    radgyr_str = str(dRadGyra)
    ecc_str = str(dEcc)
    semi_str = str(semi)
    haltenv_str = str(bHaltEnvelopeGone)
    envmass_str = str(dEnvelopeMass)
    xabs_str = str(xabs)

    # star.in strings
    mass_star_str = str(dMass_star)
    radgyra_star_str = str(dRadGyra_star)
    rotper_star_str = str(dRotPeriod_star)

    # vpl.in strings
    iVerbose_str = str(iVerbose)
    bOverwrite_str = str(bOverwrite)
    bDoLog_str = str(bDoLog)
    iDigits_str = str(iDigits)
    dMinValue_str = str(dMinValue)
    bDoForward_str = str(bDoForward)
    bVarDt_str = str(bVarDt)
    dEta_str = str(dEta)
    dStopTime_str = str(dStopTime)
    dOutputTime_str = str(dOutputTime)

    # write b.in
    b = open(runs_dir+name_idx+'/b.in','w')
    b_content = ('sName\t\t'+ name_b + 
                 '\nsaModules\t'+modules_b+
                 '\n\ndMass\t\t'+mass_str+
                 '\ndObliquity\t'+obl_str+
                 '\ndRadGyra\t'+radgyr_str+
                 '\n\ndEcc\t\t'+ecc_str+
                 '\ndSemi\t\t'+semi_str+
                 '\n\nbHaltEnvelopeGone\t'+haltenv_str+
                 '\ndEnvelopeMass\t\t'+envmass_str+
                 '\ndAtmXAbsEffH\t'+xabs_str+
                 '\n\nsaOutputOrder\t'+saOutputOrder_b+'\n')
    b.write(b_content)
    b.close()

    # write star.in
    star = open(runs_dir+name_idx+'/star.in','w')
    star_content = ('sName\t\t'+name_star+
                    '\nsaModules\t'+modules_star+
                    '\n\ndMass\t\t'+mass_star_str+
                    #'\ndRotPeriod\t'+rotper_star_str+
                    #'\ndRadGyra\t'+radgyra_star_str+
                    '\n\nsaOutputOrder\t'+saOutputOrder_star+'\n')
    star.write(star_content)
    star.close()

    vpl = open(runs_dir+name_idx+'/vpl.in','w')
    vpl_content = ('sSystemName\t'+sys_name+
                   '\niVerbose\t'+iVerbose_str+
                   '\nbOverwrite\t'+bOverwrite_str+
                   '\n\nsaBodyFiles\t'+saBodyFiles+
                   '\n\nsUnitMass\t'+sUnitMass+
                   '\nsUnitLength\t'+sUnitLength+
                   '\nsUnitTime\t'+sUnitTime+
                   '\nsUnitAngle\t'+sUnitAngle+
                   '\nsUnitTemp\t'+sUnitTemp+
                   '\n\nbDoLog\t\t'+bDoLog_str+
                   '\niDigits\t\t'+iDigits_str+
                   '\ndMinValue\t'+dMinValue_str+
                   '\n\nbDoForward\t'+bDoForward_str+
                   '\nbVarDt\t\t'+bVarDt_str+
                   '\ndEta\t\t'+dEta_str+
                   '\ndStopTime\t'+dStopTime_str+
                   '\ndOutputTime\t'+ dOutputTime_str+'\n')
    vpl.write(vpl_content)
    vpl.close()

    n+=1